<a href="https://colab.research.google.com/github/tatsuhiko-suyama/Something-/blob/main/2_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import minimize, linprog
from itertools import combinations
from functools import lru_cache

# ---------------------------
# 1. correlation 行列生成関数と共分散行列の計算
# ---------------------------
def generate_correlation(n, rho):
    """ベクトル化した相関行列生成"""
    i, j = np.indices((n, n))
    return rho ** np.abs(i - j)

def correlation_to_covariance(R_corr, sigma):
    """行列演算で共分散行列を効率的に計算"""
    D = np.diag(sigma)
    return D @ R_corr @ D

# ---------------------------
# 2. 重み付き共分散行列 V^w の計算
# ---------------------------
def compute_weighted_covariance(V_list, r_list, w):
    """
    重み付き共分散行列 V^w を計算する。

    Parameters:
      V_list : 共分散行列のリスト [V1, V2, ..., VM]
      r_list : リターンベクトルのリスト [r1, r2, ..., rM]
      w      : モデルウェイト [w1, w2, ..., wM]

    Returns:
      V_w    : 重み付き共分散行列
    """
    M = len(V_list)
    n = V_list[0].shape[0]

    # Σ w_m V_m
    V_w = np.zeros((n, n))
    for m in range(M):
        V_w += w[m] * V_list[m]

    # Σ w_m r_m
    r_w = np.zeros(n)
    for m in range(M):
        r_w += w[m] * r_list[m]

    # (Σ w_m r_m)(Σ w_m r_m)^T
    V_w -= np.outer(r_w, r_w)

    return V_w

# ---------------------------
# 3. 分岐全探索による解法（完全な実装）
# ---------------------------
def solve_portfolio_branch_M3(V_w, r1, r2, r3, mu, tol=1e-8):
    """
    M=3 の場合の平均分散ポートフォリオ問題を分岐全探索で解く。
    """
    n = V_w.shape[0]
    invV = np.linalg.inv(V_w)
    candidates = []

    # 小ヘルパー関数: pi = 0.5 * invV (sum_i lambda_i * r_i)
    def pi_from_lagrange(lmbd_vec, r_list):
        # lmbd_vec: [lambda1, lambda2, ...] (活性の分だけ)
        # r_list:   [r1, r2, ...] (活性の分だけ)
        comb = np.zeros(n)
        for lam, r_ in zip(lmbd_vec, r_list):
            comb += lam * r_
        return 0.5 * (invV @ comb)

    # ---------------------------
    # Case 1: r1, r2, r3 全て活性
    # ---------------------------
    A3 = np.array([
        [r1.T @ invV @ r1, r1.T @ invV @ r2, r1.T @ invV @ r3],
        [r2.T @ invV @ r1, r2.T @ invV @ r2, r2.T @ invV @ r3],
        [r3.T @ invV @ r1, r3.T @ invV @ r2, r3.T @ invV @ r3]
    ])
    b3 = 2 * mu * np.ones(3)
    try:
        lam3 = np.linalg.solve(A3, b3)  # lambda1, lambda2, lambda3
        if all(l > tol for l in lam3):
            pi_case1 = pi_from_lagrange(lam3, [r1, r2, r3])
            # すべて等号を満たすか確認
            c1 = r1.T @ pi_case1
            c2 = r2.T @ pi_case1
            c3 = r3.T @ pi_case1
            if (abs(c1 - mu) < tol) and (abs(c2 - mu) < tol) and (abs(c3 - mu) < tol):
                obj1 = pi_case1.T @ V_w @ pi_case1
                candidates.append(('Case1', pi_case1, obj1))
    except np.linalg.LinAlgError:
        pass  # 行列が特異ならスキップ

    # ---------------------------
    # Case 2: r1, r2 活性; r3 非活性
    # ---------------------------
    A2_12 = np.array([
        [r1.T @ invV @ r1, r1.T @ invV @ r2],
        [r2.T @ invV @ r1, r2.T @ invV @ r2]
    ])
    b2_12 = 2 * mu * np.ones(2)
    try:
        lam2_12 = np.linalg.solve(A2_12, b2_12)  # lambda1, lambda2
        if all(l > tol for l in lam2_12):
            pi_case2 = pi_from_lagrange(lam2_12, [r1, r2])
            # r1, r2 は等号 -> check
            c1 = r1.T @ pi_case2
            c2 = r2.T @ pi_case2
            c3 = r3.T @ pi_case2
            if (abs(c1 - mu) < tol) and (abs(c2 - mu) < tol) and (c3 > mu + tol):
                obj2 = pi_case2.T @ V_w @ pi_case2
                candidates.append(('Case2', pi_case2, obj2))
    except np.linalg.LinAlgError:
        pass

    # ---------------------------
    # Case 3: r1, r3 活性; r2 非活性
    # ---------------------------
    A2_13 = np.array([
        [r1.T @ invV @ r1, r1.T @ invV @ r3],
        [r3.T @ invV @ r1, r3.T @ invV @ r3]
    ])
    b2_13 = 2 * mu * np.ones(2)
    try:
        lam2_13 = np.linalg.solve(A2_13, b2_13)  # lambda1, lambda3
        if all(l > tol for l in lam2_13):
            pi_case3 = pi_from_lagrange(lam2_13, [r1, r3])
            c1 = r1.T @ pi_case3
            c2 = r2.T @ pi_case3
            c3 = r3.T @ pi_case3
            if (abs(c1 - mu) < tol) and (c2 > mu + tol) and (abs(c3 - mu) < tol):
                obj3 = pi_case3.T @ V_w @ pi_case3
                candidates.append(('Case3', pi_case3, obj3))
    except np.linalg.LinAlgError:
        pass

    # ---------------------------
    # Case 4: r2, r3 活性; r1 非活性
    # ---------------------------
    A2_23 = np.array([
        [r2.T @ invV @ r2, r2.T @ invV @ r3],
        [r3.T @ invV @ r2, r3.T @ invV @ r3]
    ])
    b2_23 = 2 * mu * np.ones(2)
    try:
        lam2_23 = np.linalg.solve(A2_23, b2_23)  # lambda2, lambda3
        if all(l > tol for l in lam2_23):
            pi_case4 = pi_from_lagrange(lam2_23, [r2, r3])
            c1 = r1.T @ pi_case4
            c2 = r2.T @ pi_case4
            c3 = r3.T @ pi_case4
            if (c1 > mu + tol) and (abs(c2 - mu) < tol) and (abs(c3 - mu) < tol):
                obj4 = pi_case4.T @ V_w @ pi_case4
                candidates.append(('Case4', pi_case4, obj4))
    except np.linalg.LinAlgError:
        pass

    # ---------------------------
    # Case 5: r1 のみ活性; r2, r3 非活性
    # ---------------------------
    denom1 = r1.T @ invV @ r1
    if abs(denom1) > tol:
        pi_case5 = (mu * (invV @ r1)) / denom1
        c1 = r1.T @ pi_case5
        c2 = r2.T @ pi_case5
        c3 = r3.T @ pi_case5
        # r1=mu, r2>mu, r3>mu
        if abs(c1 - mu) < tol and (c2 > mu + tol) and (c3 > mu + tol):
            obj5 = pi_case5.T @ V_w @ pi_case5
            candidates.append(('Case5', pi_case5, obj5))

    # ---------------------------
    # Case 6: r2 のみ活性; r1, r3 非活性
    # ---------------------------
    denom2 = r2.T @ invV @ r2
    if abs(denom2) > tol:
        pi_case6 = (mu * (invV @ r2)) / denom2
        c1 = r1.T @ pi_case6
        c2 = r2.T @ pi_case6
        c3 = r3.T @ pi_case6
        if (c1 > mu + tol) and abs(c2 - mu) < tol and (c3 > mu + tol):
            obj6 = pi_case6.T @ V_w @ pi_case6
            candidates.append(('Case6', pi_case6, obj6))

    # ---------------------------
    # Case 7: r3 のみ活性; r1, r2 非活性
    # ---------------------------
    denom3 = r3.T @ invV @ r3
    if abs(denom3) > tol:
        pi_case7 = (mu * (invV @ r3)) / denom3
        c1 = r1.T @ pi_case7
        c2 = r2.T @ pi_case7
        c3 = r3.T @ pi_case7
        if (c1 > mu + tol) and (c2 > mu + tol) and abs(c3 - mu) < tol:
            obj7 = pi_case7.T @ V_w @ pi_case7
            candidates.append(('Case7', pi_case7, obj7))

    if len(candidates) == 0:
        return None, None

    # 最小の目的関数値をもつ候補を返す
    best_candidate = min(candidates, key=lambda x: x[2])
    return best_candidate, candidates

# ---------------------------
# 4. SLSQP を用いた解法
# ---------------------------
def solve_portfolio_M3_solver(V_w, R, mu, tol=1e-8):
    """
    M=3 の場合の平均分散ポートフォリオ問題を SLSQP を用いて解く。
    """
    n = V_w.shape[0]

    def objective(pi):
        return pi @ V_w @ pi

    def grad_objective(pi):
        return 2 * V_w @ pi

    # 不等式制約をまとめて定義
    def constraints_fun(pi):
        return R @ pi - mu

    def constraints_jac(pi):
        return R

    constraints = [{
        'type': 'ineq',
        'fun': constraints_fun,
        'jac': constraints_jac
    }]

    # 初期点を linprog で求める
    A_ub = -R
    b_ub = -mu * np.ones(R.shape[0])
    bounds = [(None, None)] * n
    res_lp = linprog(c=np.zeros(n), A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')
    pi0 = res_lp.x if res_lp.success else np.ones(n)

    sol = minimize(objective, pi0, jac=grad_objective,
                   constraints=constraints, method='SLSQP', tol=tol)
    return sol

# ---------------------------
# 5. キャッシュ機能付き内側の最小化問題ソルバー
# ---------------------------
@lru_cache(maxsize=1024)
def solve_inner_min_problem_cached(w_tuple, mu, V_list_id, r_list_id):
    """
    キャッシュ機能付きの内側の最小化問題ソルバー

    Parameters:
      w_tuple   : モデルウェイト（タプル型、キャッシュのため）
      mu        : 目標リターン
      V_list_id : V_listのIDハッシュ（キャッシュのため）
      r_list_id : r_listのIDハッシュ（キャッシュのため）
    """
    global V_list_global, r_list_global
    w = np.array(w_tuple)

    # グローバル変数から取得
    V_list = V_list_global
    r_list = r_list_global

    # 重み付き共分散行列 V^w を計算
    V_w = compute_weighted_covariance(V_list, r_list, w)

    # リターン行列 R を作成
    R_matrix = np.vstack(r_list)

    # 内側の最小化問題を解く
    sol = solve_portfolio_M3_solver(V_w, R_matrix, mu)

    if sol.success:
        return sol.x, sol.fun, True
    else:
        return None, np.inf, False

def solve_inner_min_problem(V_list, r_list, w, mu):
    """キャッシュ機能を使う内側の最小化問題のラッパー"""
    global V_list_global, r_list_global

    # グローバル変数に保存
    V_list_global = V_list
    r_list_global = r_list

    # IDを生成
    V_list_id = id(tuple(map(id, V_list)))
    r_list_id = id(tuple(map(id, r_list)))

    return solve_inner_min_problem_cached(tuple(w), mu, V_list_id, r_list_id)

# ---------------------------
# 6. 効率化した頂点と辺の探索
# ---------------------------
def solve_maxmin_vertices_edges(V_list, r_list, mu):
    """
    効率化された単体の頂点と辺探索
    """
    M = len(V_list)

    # 1. すべての頂点を一度に処理
    vertices = np.eye(M)  # 単位行列で頂点を表現
    vertex_results = []

    for m, w in enumerate(vertices):
        pi, obj, success = solve_inner_min_problem(V_list, r_list, w, mu)
        if success:
            vertex_results.append({
                'type': 'vertex',
                'w': w.copy(),
                'pi': pi,
                'obj': obj,
                'success': success,
                'vertex_idx': m
            })

    # 2. 辺上の点をベクトル化して処理
    edge_results = []
    alphas = np.linspace(0.1, 0.9, 9)  # 辺上の探索点

    for i, j in combinations(range(M), 2):
        # すべてのαに対して一括で処理
        for alpha in alphas:
            w = np.zeros(M)
            w[i] = alpha
            w[j] = 1 - alpha

            pi, obj, success = solve_inner_min_problem(V_list, r_list, w, mu)

            if success:
                edge_results.append({
                    'type': 'edge',
                    'w': w.copy(),
                    'pi': pi,
                    'obj': obj,
                    'success': success,
                    'edge': (i, j),
                    'alpha': alpha
                })

    # 3. 結果を結合して最適値を探索
    results = vertex_results + edge_results

    valid_results = [r for r in results if r['success']]
    if not valid_results:
        return None, None, np.nan, results

    best_result = max(valid_results, key=lambda r: r['obj'])

    return best_result['w'], best_result['pi'], best_result['obj'], results

# ---------------------------
# 7. M次元ベクトルのw最適化関数
# ---------------------------
def solve_maxmin_direct_optimization(V_list, r_list, mu, max_iter=10):
    """
    直接最適化手法を用いてMaxMin問題を解く関数。
    任意のM次元のwベクトルに対応。

    Parameters:
      V_list  : 共分散行列のリスト [V1, V2, ..., VM]
      r_list  : リターンベクトルのリスト [r1, r2, ..., rM]
      mu      : 目標リターン
      max_iter: 最大反復回数（異なる初期値での反復）

    Returns:
      w_opt   : 最適なモデルウェイト
      pi_opt  : 最適なポートフォリオ
      obj_val : 最適値
      results : 最適化の詳細結果
    """
    M = len(V_list)

    # 内側の最小化問題の関数
    def neg_inner_min_obj(w):
        """最大化問題に変換するため、符号を反転した内側の目的関数"""
        pi, obj, success = solve_inner_min_problem(V_list, r_list, w, mu)
        if success:
            return -obj  # 最大化するために符号を反転
        else:
            return 1e10  # 失敗した場合は大きなペナルティ

    # 単体制約の設定
    bounds = [(0, 1)] * M  # 各wは0から1の範囲

    # 合計が1になる等式制約
    constraints = [{
        'type': 'eq',
        'fun': lambda w: np.sum(w) - 1,
        'jac': lambda w: np.ones(M)
    }]

    # 最良の結果を保持
    best_w = None
    best_pi = None
    best_obj = -np.inf
    all_results = []

    # 複数の初期点から最適化を実行
    for _ in range(max_iter):
        # ランダムな初期点（単体上の点）
        w0 = np.random.rand(M)
        w0 /= np.sum(w0)  # 合計が1になるように正規化

        # 最適化を実行
        sol = minimize(
            neg_inner_min_obj,
            w0,
            bounds=bounds,
            constraints=constraints,
            method='SLSQP',
            options={'disp': False, 'maxiter': 1000}
        )

        if sol.success:
            # 内側の問題を解いて最適なポートフォリオを取得
            w_opt = sol.x
            pi_opt, obj_opt, success = solve_inner_min_problem(V_list, r_list, w_opt, mu)

            all_results.append({
                'w': w_opt,
                'pi': pi_opt,
                'obj': obj_opt,
                'success': success,
                'initial_w': w0
            })

            # よりよい解が見つかったら更新
            if success and obj_opt > best_obj:
                best_w = w_opt
                best_pi = pi_opt
                best_obj = obj_opt

    # 結果を返す
    return best_w, best_pi, best_obj, all_results

# ---------------------------
# 8. M次元単体上でのランダム探索による最適化
# ---------------------------
def solve_maxmin_random_search(V_list, r_list, mu, n_samples=1000):
    """
    M次元単体上でのランダム探索によりMaxMin問題を解く関数。

    Parameters:
      V_list   : 共分散行列のリスト [V1, V2, ..., VM]
      r_list   : リターンベクトルのリスト [r1, r2, ..., rM]
      mu       : 目標リターン
      n_samples: ランダムサンプルの数

    Returns:
      w_opt    : 最適なモデルウェイト
      pi_opt   : 最適なポートフォリオ
      obj_val  : 最適値
      results  : 探索結果
    """
    M = len(V_list)

    # ランダムなM次元単体上の点を生成
    def generate_simplex_points(n):
        """ディリクレ分布を使用してM次元単体上の点を効率的に生成"""
        # M次元の単位ガンマ乱数から単体上の点を生成（非常に効率的）
        points = np.random.gamma(shape=1.0, scale=1.0, size=(n, M))
        # 単体上の点に正規化
        return points / np.sum(points, axis=1)[:, np.newaxis]

    # ランダム点を生成
    w_samples = generate_simplex_points(n_samples)

    # 結果を格納する配列を初期化
    obj_vals = np.zeros(n_samples)
    pi_solutions = [None] * n_samples
    success_flags = np.zeros(n_samples, dtype=bool)

    # すべてのサンプルポイントで内側の最小化問題を解く
    for i, w in enumerate(w_samples):
        pi, obj, success = solve_inner_min_problem(V_list, r_list, w, mu)
        obj_vals[i] = obj
        pi_solutions[i] = pi
        success_flags[i] = success

        # 進捗表示（オプション）
        if (i + 1) % 100 == 0:
            print(f"  ランダム探索: {i+1}/{n_samples} 完了")

    # 成功した中で目的関数値が最大のインデックスを見つける
    valid_indices = np.where(success_flags)[0]
    if len(valid_indices) == 0:
        return None, None, np.nan, {}

    best_idx = valid_indices[np.argmax(obj_vals[valid_indices])]

    results = {
        'w_samples': w_samples,
        'obj_vals': obj_vals,
        'pi_solutions': pi_solutions,
        'success_flags': success_flags
    }

    return w_samples[best_idx], pi_solutions[best_idx], obj_vals[best_idx], results

# ---------------------------
# 9. 一般的なM次元単体上での探索
# ---------------------------
def solve_maxmin_comprehensive(V_list, r_list, mu, random_samples=1000, direct_opt_iters=5):
    """
    複数の方法を組み合わせた包括的なMaxMin問題解法。
    任意のM次元に対応。

    Parameters:
      V_list         : 共分散行列のリスト [V1, V2, ..., VM]
      r_list         : リターンベクトルのリスト [r1, r2, ..., rM]
      mu             : 目標リターン
      random_samples : ランダム探索のサンプル数
      direct_opt_iters: 直接最適化の繰り返し回数

    Returns:
      best_method    : 最良の結果を得た手法
      w_opt          : 最適なモデルウェイト
      pi_opt         : 最適なポートフォリオ
      obj_val        : 最適値
    """
    M = len(V_list)
    results = {}

    # 1. 頂点探索（任意のMに対応）
    print("M次元の頂点を探索中...")
    vertices = np.eye(M)
    vertex_results = []

    for m, w in enumerate(vertices):
        pi, obj, success = solve_inner_min_problem(V_list, r_list, w, mu)
        if success:
            vertex_results.append({
                'type': 'vertex',
                'w': w.copy(),
                'pi': pi,
                'obj': obj,
                'success': success,
                'vertex_idx': m
            })

    # 最良の頂点結果
    if vertex_results:
        best_vertex = max(vertex_results, key=lambda r: r['obj'])
        results['vertex'] = {
            'w': best_vertex['w'],
            'pi': best_vertex['pi'],
            'obj': best_vertex['obj'],
            'details': vertex_results
        }
    else:
        results['vertex'] = None

    # 2. 辺探索（任意のMに対応）
    print("M次元の辺を探索中...")
    edge_results = []
    alphas = np.linspace(0.1, 0.9, 9)

    for i, j in combinations(range(M), 2):
        for alpha in alphas:
            w = np.zeros(M)
            w[i] = alpha
            w[j] = 1 - alpha

            pi, obj, success = solve_inner_min_problem(V_list, r_list, w, mu)

            if success:
                edge_results.append({
                    'type': 'edge',
                    'w': w.copy(),
                    'pi': pi,
                    'obj': obj,
                    'success': success,
                    'edge': (i, j),
                    'alpha': alpha
                })

    # 最良の辺結果
    if edge_results:
        best_edge = max(edge_results, key=lambda r: r['obj'])
        results['edge'] = {
            'w': best_edge['w'],
            'pi': best_edge['pi'],
            'obj': best_edge['obj'],
            'details': edge_results
        }
    else:
        results['edge'] = None

    # 3. ランダム探索
    print(f"M次元単体上でランダム探索中 ({random_samples}サンプル)...")
    rand_w_opt, rand_pi_opt, rand_obj_val, rand_details = solve_maxmin_random_search(
        V_list, r_list, mu, n_samples=random_samples)

    if rand_w_opt is not None:
        results['random'] = {
            'w': rand_w_opt,
            'pi': rand_pi_opt,
            'obj': rand_obj_val,
            'details': rand_details
        }
    else:
        results['random'] = None

    # 4. 直接最適化
    print(f"直接最適化法で探索中 ({direct_opt_iters}回の繰り返し)...")
    direct_w_opt, direct_pi_opt, direct_obj_val, direct_details = solve_maxmin_direct_optimization(
        V_list, r_list, mu, max_iter=direct_opt_iters)

    if direct_w_opt is not None:
        results['direct'] = {
            'w': direct_w_opt,
            'pi': direct_pi_opt,
            'obj': direct_obj_val,
            'details': direct_details
        }
    else:
        results['direct'] = None

    # 5. 最良の結果を選択
    valid_methods = [method for method, res in results.items() if res is not None]
    if not valid_methods:
        return None, None, None, np.nan

    best_method = max(valid_methods, key=lambda m: results[m]['obj'])
    best_result = results[best_method]

    return best_method, best_result['w'], best_result['pi'], best_result['obj']

# ---------------------------
# 10. M次元単体上の結果可視化（主成分分析を使用）
# ---------------------------
def plot_m_dim_results(w_samples, obj_vals, success_flags, best_w=None, method="PCA"):
    """
    M次元単体上の結果を2次元で可視化する。
    Principal Component Analysis (PCA)を使用して次元削減を行う。

    Parameters:
      w_samples    : サンプル点のリスト
      obj_vals     : 各サンプル点での目的関数値
      success_flags: 各サンプル点での最適化成功フラグ
      best_w       : 最適なモデルウェイト（あれば表示）
      method       : 可視化手法（"PCA"または"TSNE"）
    """
    try:
        from sklearn.decomposition import PCA
        from sklearn.manifold import TSNE
    except ImportError:
        print("Warning: scikit-learn is not installed. PCA/TSNE visualization is not available.")
        return None

    # 成功した点のみ抽出
    valid_indices = np.where(success_flags)[0]
    if len(valid_indices) == 0:
        print("可視化するための有効なデータがありません")
        return None

    valid_w = w_samples[valid_indices]
    valid_obj = obj_vals[valid_indices]

    # 次元削減を行う
    if method == "PCA":
        reducer = PCA(n_components=2)
    else:  # TSNE
        reducer = TSNE(n_components=2, perplexity=min(30, len(valid_w)-1))

    w_2d = reducer.fit_transform(valid_w)

    # 散布図をプロット
    fig, ax = plt.subplots(figsize=(10, 8))
    scatter = ax.scatter(w_2d[:, 0], w_2d[:, 1], c=valid_obj,
                         cmap='viridis', alpha=0.8, s=50, edgecolors='w')
    plt.colorbar(scatter, label='Objective Value')

    # 最適点をプロット
    if best_w is not None:
        # 最適点も同じ次元削減で変換
        best_w_2d = reducer.transform(best_w.reshape(1, -1))
        ax.plot(best_w_2d[0, 0], best_w_2d[0, 1], 'ro', markersize=10)
        ax.text(best_w_2d[0, 0], best_w_2d[0, 1], 'Optimal',
                color='red', ha='center', va='bottom')

    ax.set_title(f'M次元単体上の目的関数値 ({method}による可視化)')
    ax.set_xlabel('Dimension 1')
    ax.set_ylabel('Dimension 2')
    plt.tight_layout()

    return fig

# ---------------------------
# 11. メイン処理
# ---------------------------
if __name__ == "__main__":
    # グローバル変数の初期化（キャッシュのため）
    V_list_global = None
    r_list_global = None

    # 固定パラメータ
    n = 5  # 資産数
    sigma = np.array([0.2, 0.25, 0.30, 0.35, 0.40])  # 標準偏差

    # まずM = 3のケースで実行する
    print("=== M = 3の場合のMaxMin問題 ===")
    M = 3

    # モデルの設定 (M=3)
    rho_list = [0.3, 0.5, 0.7]  # 各モデルの相関パラメータ

    # リターンベクトル
    r1 = np.array([0.02, 0.04, 0.06, 0.08, 0.10])
    r2 = np.array([0.01, 0.03, 0.05, 0.07, 0.09])
    r3 = np.array([0.015, 0.035, 0.055, 0.075, 0.095])
    r_list = [r1, r2, r3]

    # 各モデルの共分散行列を一括生成
    V_list = []
    for rho in rho_list:
        correlation = generate_correlation(n, rho)
        V = correlation_to_covariance(correlation, sigma)
        V_list.append(V)

    # 目標リターンの設定
    mu_list = [0.01, 0.05, 0.10]

    # 結果を格納するリスト
    all_results_M3 = []

    for mu in mu_list:
        print(f"======================================================")
        print(f"目標リターン mu = {mu}")

        # 頂点と辺のみ探索によるMaxMin問題の解法
        print("\n【頂点と辺のみ探索によるMaxMin問題の解法】")
        ve_w_opt, ve_pi_opt, ve_obj_val, ve_results = solve_maxmin_vertices_edges(
            V_list, r_list, mu)

        if ve_w_opt is not None:
            print(f"最適なモデルウェイト w* = {ve_w_opt}")
            print(f"最適なポートフォリオ π* = {ve_pi_opt}")
            print(f"最適値 (MaxMin) = {ve_obj_val}")

            # リターン制約の確認（行列演算で効率化）
            returns = np.array([r @ ve_pi_opt for r in r_list])
            for i, ret in enumerate(returns):
                print(f"モデル {i+1} のリターン: {ret:.6f} (>= {mu})")

            # どの頂点/辺で最適か
            best_result = max([r for r in ve_results if r['success']], key=lambda r: r['obj'])
            if best_result['type'] == 'vertex':
                model_idx = np.argmax(best_result['w'])
                print(f"最適解はモデル {model_idx+1} の頂点にあります")
            else:  # edge
                i, j = best_result['edge']
                alpha = best_result['alpha']
                print(f"最適解はモデル {i+1} とモデル {j+1} の間の辺上にあります (α = {alpha:.2f})")
        else:
            print("有効な解が見つかりませんでした。")

        # 結果を保存
        result = {
            'mu': mu,
            've_w_opt': ve_w_opt,
            've_pi_opt': ve_pi_opt,
            've_obj_val': ve_obj_val
        }
        all_results_M3.append(result)

    # 結果の要約 (M=3)
    summary_df_M3 = pd.DataFrame([
        {
            'mu': res['mu'],
            'obj_val': res['ve_obj_val'],
            'w_opt': str(np.round(res['ve_w_opt'], 3)) if res['ve_w_opt'] is not None else None
        }
        for res in all_results_M3
    ])

    print("\n======================================================")
    print("M=3のMaxMin問題の解法結果の要約:")
    print(summary_df_M3)

    # M = 4のケースで実行する
    print("\n\n=== M = 4の場合のMaxMin問題 ===")
    M = 4

    # モデルの設定 (M=4)
    rho_list = [0.2, 0.4, 0.6, 0.8]  # 各モデルの相関パラメータ

    # リターンベクトル (M=4)
    r1 = np.array([0.02, 0.04, 0.06, 0.08, 0.10])
    r2 = np.array([0.01, 0.03, 0.05, 0.07, 0.09])
    r3 = np.array([0.015, 0.035, 0.055, 0.075, 0.095])
    r4 = np.array([0.025, 0.045, 0.065, 0.085, 0.105])  # 4つ目のモデル
    r_list = [r1, r2, r3, r4]

    # 各モデルの共分散行列を一括生成
    V_list = []
    for rho in rho_list:
        correlation = generate_correlation(n, rho)
        V = correlation_to_covariance(correlation, sigma)
        V_list.append(V)

    # 結果を格納するリスト
    all_results_M4 = []

    for mu in mu_list:
        print(f"======================================================")
        print(f"目標リターン mu = {mu}")

        # 包括的な解法を実行
        print("\n【M次元単体上でのMaxMin問題解法】")
        best_method, w_opt, pi_opt, obj_val = solve_maxmin_comprehensive(
            V_list, r_list, mu, random_samples=500, direct_opt_iters=3)

        if w_opt is not None:
            print(f"最適化手法: {best_method}")
            print(f"最適なモデルウェイト w* = {w_opt}")
            print(f"最適なポートフォリオ π* = {pi_opt}")
            print(f"最適値 (MaxMin) = {obj_val}")

            # リターン制約の確認（行列演算で効率化）
            returns = np.array([r @ pi_opt for r in r_list])
            for i, ret in enumerate(returns):
                print(f"モデル {i+1} のリターン: {ret:.6f} (>= {mu})")
        else:
            print("有効な解が見つかりませんでした。")

        # ランダム探索と可視化
        print("\n【ランダム探索のみによるMaxMin問題解法】")
        rand_w_opt, rand_pi_opt, rand_obj_val, rand_results = solve_maxmin_random_search(
            V_list, r_list, mu, n_samples=1000)

        if rand_w_opt is not None:
            # M次元可視化
            try:
                fig = plot_m_dim_results(
                    rand_results['w_samples'],
                    rand_results['obj_vals'],
                    rand_results['success_flags'],
                    best_w=w_opt,
                    method="PCA"
                )
                if fig:
                    plt.savefig(f"maxmin_M{M}_mu_{mu}_PCA.png")
                    plt.close()
            except Exception as e:
                print(f"可視化中にエラーが発生しました: {e}")

        # 結果を保存
        result = {
            'mu': mu,
            'best_method': best_method,
            'w_opt': w_opt,
            'pi_opt': pi_opt,
            'obj_val': obj_val
        }
        all_results_M4.append(result)

    # 結果の要約 (M=4)
    summary_df_M4 = pd.DataFrame([
        {
            'mu': res['mu'],
            'best_method': res['best_method'],
            'obj_val': res['obj_val'],
            'w_opt': str(np.round(res['w_opt'], 3)) if res['w_opt'] is not None else None
        }
        for res in all_results_M4
    ])

    print("\n======================================================")
    print("M=4のMaxMin問題の解法結果の要約:")
    print(summary_df_M4)